## Part 2: 分层线性模型(Multilevel linear model)

本节的目的在与：了解如何贝叶斯分层广义线性模型 (Bayesian Multilevel Generalized linear model)如何解释数据的分层结构。

重点在于：
- 了解分层线性模型的基本概念：包括随机截距与随机斜率。
- 通过 PyMc 分别拟合随机截距模型与随机斜率模型(包含随机截距)。

In [1]:
#加载需要使用的库
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm

np.random.seed(123)  # 随机数种子，确保随后生成的随机数相同

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


### 回顾：广义线性模型GLM

**当因变量为离散变量**

比如，因变量为二分变量 (答题正确率)，其中1代表回答正确，0代表回答错误。

正确率为离散变量，服从伯努利(Bernoulli)分布。

![Image Name](https://cdn.kesci.com/upload/image/rloa62fn8w.png?imageView2/0/w/960/h/960)


对于因变量为离散变量的情况，我们需要使用广义线性模型(Generalized linear model，GLM)。

其特点为：
- 分布簇 (dist)不再局限于正态分布，而是允许其他不同的分布，比如 $y \sim Bernoulli(p)$。
- 需要 **链接函数g()** 将 $\alpha + \beta * x$  映射到 p 所在的范围。

| 一般线性模型 | 广义线性模型 | 
|---|---|
| $y \sim Normal(\mu,sigma)$ | $y \sim dist(p)$ |
| $\mu = \alpha + \beta *x$ | $p = g(\mu)$|

链接函数的具体转化过程，以逻辑(logit)回归为例：
1. 令 $z = \alpha + \beta *x$，$\mu$的范围为 $(-\infty, +\infty)$。
2. $p = g(z)$，其中 g() 为链接函数，输出结果 p 的范围为 $(0,1)$。
3.  最后将 p 输入到分布函数中 $y \sim Bernoulli(p)$。


![Image Name](https://cdn.kesci.com/upload/image/rloa6zyf5a.png?imageView2/0/w/600/h/600)


**具体实例**

基于泰国初等教育数据库，我们想探究，学生是否接受学前教育对之后学生是否留级的影响？

数据来源：泰国初等教育的全国调查 (Raudenbush & Bhumirat，1992)。
- 数据中的每一行表示一名学生。
- 因变量 `REPEAT` 为二分类变量，表示学生在初等教育期间是否留级，1 = 留级，0 = 非留级。
- 自变量为学生是否受过学前教育 `PPED` (0 = 否，1 = 是)。

数据来源：https://github.com/MultiLevelAnalysis/Datasets-third-edition-Multilevel-book/blob/master/chapter%206/Thaieduc/thaieduc.sav

In [2]:
# 加载数据
data = pd.read_csv("./thaieduc.csv")
data.PPED = data.PPED.map({'yes':1,'no':0})

In [3]:
data.head()

,SCHOOLID,SEX,PPED,REPEAT,MSESC
0,10103,girl,0,0,0.88
1,10103,girl,0,0,0.88
2,10103,girl,1,0,0.88
3,10103,girl,1,0,0.88
4,10103,girl,1,0,0.88


回顾：在PyMc中建立GLM的方法

- x 为自变量：学生是否受过学前教育 `PPED` (0 = 否，1 = 是)。
- y 为因变量：学生在初等教育期间是否留级 `REPEAT` (1 = 留级，0 = 非留级)。
- `p = alpha + beta * x` 为模型线性表达式。
- `Bernoulli` 为GLM的分布簇。
- `logit_p=p` 为GLM的链接函数。

In [4]:
with pm.Model() as GLM_model:
    # 定义先验
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1,shape=1)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['PPED'])
    # 线性模型：p是确定性随机变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha + beta * x)
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data["REPEAT"])

### 什么是分层线性模型




分层线性模型/多水平线性模型（Multilevel Linear Model，MLM）：关注“多层嵌套数据”，在一个以上层次上变化参数的线性模型。

其他近义词：
- 分层线性模型（Hierarchical Linear Model, HLM）
- 线性混合模型（Linear Mixed Model, LMM）
- 混合效应模型（Mixed Effects Model）
- 随机效应模型（Random Effects Model）
- 随机系数模型（Random Coefficients Model）
- 增长曲线模型（Growth Curve Model）
- ……

例子：考察员工工作年限 (experience)是否能预测员工工资 (salary)

![Image Name](https://cdn.kesci.com/upload/image/rmani77u3t.png?imageView2/0/w/640/h/640)


但员工可能来自不同的学校

此时存在两个分析水平
- Level 1 为 population水平，代表每个学校内员工经验对工资的影响。
- Level 2 为 group水平，代表不同学校的平均效应。

![Image Name](https://cdn.kesci.com/upload/image/rmdc4d1nk0.png?imageView2/0/w/640/h/640)

员工工作年限 (experience)对员工工资 (salary)的预测作用受不同学校的影响

- 不同学校员工工资存在不同。
	- 这被称为随机截距，截距的变化代表不同学校间员工平均工资的差异。
	- ![Image Name](https://cdn.kesci.com/upload/image/rmanj9sw62.png?imageView2/0/w/640/h/640)
- 不同学校中，员工工作年限对工资的预测作用会发生变化。
	- 这被称为随机斜率，斜率在不同学校间的变化代表，工作年限对工资的预测作用会在不同学校将发生变化。
	- ![Image Name](https://cdn.kesci.com/upload/image/rmank74ytm.png?imageView2/0/w/640/h/640)
- 随机截距 + 随机斜率
	- ![Image Name](https://cdn.kesci.com/upload/image/rmankk5jgh.png?imageView2/0/w/640/h/640)

图片来源：http://mfviz.com/hierarchical-models/

随机截距与随机会随着不同学校变化，表达式如下：


![Image Name](https://cdn.kesci.com/upload/image/rmb2ix212p.png?imageView2/0/w/640/h/640)


### Workflow

在回顾广义线性模型 (GLM)与介绍分层线性模型 (MLM/HLM)的基础知识后，我们回到泰国初等教育的实例，并通过PyMC完成的贝叶斯分层广义线性模型建模的全过程 (full workflow)。

![Image Name](https://cdn.kesci.com/upload/image/rkvikqg9q6.png?imageView2/0/w/650/h/650)

### (1) 提出研究问题

使用泰国初等教育数据探究的研究问题为：
- 学生是否接受学前教育对学生留级的影响？
- 在考虑不同学校的分层结构时，接受学前教育对学生是否留级的影响有哪些变化？
- 分层模型是否能提供额外的信息？

### (2) 数据收集

数据来源：泰国初等教育的全国调查 (Raudenbush & Bhumirat，1992)。
- 数据中的每一行表示一名学生。
- 因变量 `REPEAT` 为二分类变量，表示学生在初等教育期间是否留级，1 = 留级，0 = 非留级。
- 自变量为学生是否受过学前教育 `PPED` (0 = 否，1 = 是)。
- 其他变量包括学生性别 `SEX`、学生所在的学校 `SCHOOLID`和学校平均社会经济地位 (SES)分数 `MSESC`。

数据来源：https://github.com/MultiLevelAnalysis/Datasets-third-edition-Multilevel-book/blob/master/chapter%206/Thaieduc/thaieduc.sav

In [42]:
# 加载数据
data = pd.read_csv("./thaieduc.csv")
data.PPED = data.PPED.map({'yes':1,'no':0})

In [6]:
data.head()

,obs_id,SCHOOLID,SEX,PPED,REPEAT,MSESC
0,0,10103,girl,0,0,0.88
1,1,10103,girl,0,0,0.88
2,2,10103,girl,1,0,0.88
3,3,10103,girl,1,0,0.88
4,4,10103,girl,1,0,0.88


#### 描述统计与探索性分析

In [7]:
# groupby 可以按照某条件对数据进行分组，之后再使用 mean函数可以实现分组求均值。
data.groupby('PPED').REPEAT.mean() 

PPED
0    0.277778
1    0.191729
Name: REPEAT, dtype: float64

可以发现，没有接受过学前教育学生的留级率 (18%)高于接受过学前教育的学生 (11%)。

可视化如下：

In [8]:
data.groupby('PPED').REPEAT.mean().plot.bar()
plt.show()

<Figure size 432x288 with 1 Axes>

对比不同学校下，学前教育对留级率的影响。

其中，每一条线代表一个学校。

可见，不同学校间的学前教育对留级率的影响存在差异。但整体存在接受学前教育后留级率下降的趋势。

In [9]:
# 计算不同学校下，是否接受学前教育的留级率
data_plot = data.groupby(["SCHOOLID",'PPED']).REPEAT.mean() 
# 根据留级率进行绘图
for schoolID,data_i in data_plot.groupby("SCHOOLID"):
    data_i.plot.line(x="PPED")
plt.xlabel("pped")             # 横坐标为是否接受学前教育
plt.xticks([0,1],["no","yes"]) # 0为否，1为是
plt.ylabel(r"REPEAT \%")       # 纵坐标为留级率

Text(0, 0.5, 'REPEAT \\%')

<Figure size 432x288 with 1 Axes>

### (3) 选择模型

由于因变量(是否留级)是二分离散变量，因此我们选择使用基于伯努利(Bernoulli)分布的广义线性模型(Generalized linear model，GLM)进行模型拟合。

这里我们考虑三种可能的模型：
- 模型1：仅包含随机截距的模型。表示拟合不同学校的平均留级率。
- 模型2：随机截距模型。在模型1的基础上，考虑学前教育在level1 (group)的总体效应，这种总体效应也成为固定效应。
- 模型3：随机斜率模型。在模型2的基础上，考虑学前教育在level2 (population)的不同学校上的不同效应，这种总体效应也成为随机效应。其实随机截距也属于随机效应。

#### 模型1：仅包含随机截距的模型

$$
y_{i, j} = \alpha_{j} + \epsilon
$$

该模型只拟合不同学校的平均留级率。
- 其中i为不同学生，j为不同学校。
- $\alpha_{j}$ 为截距，代表平均留级率。该解决随着不同学校j进行变化，因此称为随机截距。
- $\alpha_{j} = \alpha +  \epsilon$ 等价于 $\alpha_{j} \sim \text{Normal} (\alpha,  \epsilon)$。
- 其中，$\alpha$ 为 level 1 group的效应，而 $\alpha_{j}$ 为 level 2 population的效应。
- 注意，此时的模型没有考虑自变量 **学前教育** 的影响。

In [12]:
# 将数据分层变量"学校(school)"转换为因子(factor)类型
school_idxs, school = pd.factorize(data.SCHOOLID)
# 定义学校与数据的映射：即标注哪名学生(行)属于哪一所学校
coords = {
    "school": school,
    "obs_id": np.arange(len(school_idxs)),
}

In [33]:
with pm.Model(coords=coords) as model1:
    # 定义level2学校的 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=0.0, sigma=100)  # 对应上述公式
    sigma_alpha = pm.HalfNormal("sigma_alpha", 50)
    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="school")
    # 定义数据分层变量"学校(school)"
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    # 定义线性模型：p是确定性随机变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha[school_idx])
    # 定义似然函数：Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

我们通过 PyMc 自带的函数 `model_to_graphviz()` 将模型可视化。

需要注意的是：
- 我们假设 $\alpha$ (先验)服从一个正态分布 其均值为 mu_alpha，误差为 sigma_alpha。
- 并且 mu_alpha 和 sigma_alpha 各自服从不同的分布，如下图，这些分布被称为 **超先验(hyperpriors)** ，可以理解为超越先验分布的分布。
- 定义超先验的目的在于约束学校变量带来的差异，这被称为 "shrinkage"，是分层模型 partial pooling的效果。更多详情请参考 https://mc-stan.org/rstanarm/articles/pooling.html

In [14]:
pm.model_to_graphviz(model1)

#### 模型2：随机截距模型

在模型1的基础上，考虑学前教育在level1的总体效应。
$$
y_{i, j} = \alpha_{j} + \beta*\text{x}_{i, j} + \epsilon
$$

- x为自变量是否接受学前教育。
- 需要注意的是，学前教育的效应 $\beta$ 不随学校进行变化。这意味着它在不同学生和不同学校间保持固定，因此也称为**固定效应**。

In [34]:
with pm.Model(coords=coords) as model2:
    # 定义level2学校的 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=0.0, sigma=100)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 50)
    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="school")
    beta = pm.Normal('beta',mu=0,sd=10)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data.PPED, dims="obs_id")
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    # 定义线性模型：p是确定性随机变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha[school_idx] + beta * x)
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

In [17]:
pm.model_to_graphviz(model2)

#### 模型3：随机斜率模型

$$
y_{i, j} = \alpha_{j} + \beta_{j}*\text{x}_{i, j} + \epsilon
$$

在模型2的基础上，考虑学前教育在level2的不同学校上的不同效
- 此时，x的效应 $\beta_{j}$ 随不同的学校 j 进行变化。
- 这种总体效应也成为随机效应。其实随机截距也属于随机效应。


In [36]:
with pm.Model(coords=coords) as model3:
    # 定义level2学校的 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=0.0, sigma=100)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 50)
    mu_beta = pm.Normal("mu_beta", mu=0.0, sigma=100)
    sigma_beta = pm.HalfNormal("sigma_beta", 50)
    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="school")
    beta = pm.Normal('beta',mu=mu_beta,sd=sigma_beta, dims="school")
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data.PPED, dims="obs_id")
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    # 定义线性模型：p是确定性随机变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha[school_idx] + beta[school_idx] * x)
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

注意观察，每个随机效应的先验 (alpha 和 beta)都存在超先验 (hyperpriors)。

同样，beta参数的超先验约束了不同学校间学前教育效应的影响，可避免出现极端值。

In [21]:
pm.model_to_graphviz(model3)

这里我们先考虑最后一个模型 (模型3)，在后面模型比较的部分，再考虑其他两个模型。

### (4)选择先验

In [37]:
with model3:
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=500)

首先查看 p 的先验分布。

注意，这里没有用链接函数对 p 进行转换，因此其范围在 -200到200之间，而不是 0-1之间。

In [38]:
az.plot_density(
    {'p':prior_checks['p']}
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (500) than draws (338). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 432x288 with 1 Axes>

其次，查看模型截距和斜率的先验。

In [39]:
az.plot_density(
    {'alpha':prior_checks['alpha'],
    'beta':prior_checks['beta']}
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (500) than draws (19). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 993.6x331.2 with 2 Axes>

最后，别忘了查看超先验所对应超参数(hyperparameters)的先验分布。

In [40]:
az.plot_density(
    {
        'mu_alpha':prior_checks['mu_alpha'],
        'sigma_alpha':prior_checks['sigma_alpha'],
        'mu_beta':prior_checks['mu_beta'],
        'sigma_beta':prior_checks['sigma_beta']
    }
    )
plt.show()

<Figure size 1490.4x662.4 with 6 Axes>

### (5) 拟合数据

拟合数据需要注意，虽然我们主要探究的自变量为"是否接受学前教育"，但是数据分层结构变量"学校"也是非常重要的，不要忽视了。

In [41]:
data.head()

,obs_id,SCHOOLID,SEX,PPED,REPEAT,MSESC
0,0,10103,girl,0,0,0.88
1,1,10103,girl,0,0,0.88
2,2,10103,girl,1,0,0.88
3,3,10103,girl,1,0,0.88
4,4,10103,girl,1,0,0.88


In [44]:
data.groupby('SCHOOLID').REPEAT.mean().plot.bar()
plt.show()

<Figure size 432x288 with 1 Axes>

In [45]:
# 将数据分层变量"学校(school)"转换为因子(factor)类型
school_idxs, school = pd.factorize(data.SCHOOLID)
# 定义学校与数据的映射：即标注哪名学生(行)属于哪一所学校
coords = {
    "school": school,
    "obs_id": np.arange(len(school_idxs)),
}

with pm.Model(coords=coords) as model3:
    # 定义level2学校的 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=0.0, sigma=100)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 50)
    mu_beta = pm.Normal("mu_beta", mu=0.0, sigma=100)
    sigma_beta = pm.HalfNormal("sigma_beta", 50)
    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="school")
    beta = pm.Normal('beta',mu=mu_beta,sd=sigma_beta, dims="school")
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data.PPED, dims="obs_id")
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    # 定义线性模型：p是确定性随机变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha[school_idx] + beta[school_idx] * x)
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

### (6)采样过程诊断

如果使用MCMC对后验进行近似，则需要首先对MCMC过程进行评估。

* 是否收敛；
* 是否接近真实的后验。

对采样过程的评估我们会采用目视检查或rhat这个指标

In [56]:
with model3:
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace3 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma_beta, mu_beta, sigma_alpha, mu_alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 22 seconds.
There were 117 divergences after tuning. Increase `target_accept` or reparameterize.
There were 201 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8112369868199605, but should be close to 0.9. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.


0, dim: obs_id, 338 =? 338


In [49]:
az.plot_trace(trace3, var_names=['alpha','beta'])
plt.show()

<Figure size 864x288 with 4 Axes>

注意，由于参数会随着学校进行变化，因此每一所学校(19所学校)都对应一个参数，如下：

In [50]:
az.summary(trace3, var_names=['alpha','beta'], kind="diagnostics")

,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],0.025,0.018,444.0,1200.0,1.03
alpha[1],0.019,0.013,409.0,1320.0,1.01
alpha[2],0.020,0.015,535.0,912.0,1.01
alpha[3],0.042,0.031,113.0,909.0,1.03
alpha[4],0.035,0.025,255.0,970.0,1.04
alpha[5],0.018,0.013,388.0,1752.0,1.01
alpha[6],0.033,0.025,127.0,1662.0,1.02
alpha[7],0.031,0.022,340.0,1036.0,1.03
alpha[8],0.016,0.012,469.0,1414.0,1.01
alpha[9],0.018,0.013,616.0,1353.0,1.01


### (7)模型诊断

在MCMC有效的前提下，需要继续检验模型是否能够较好地拟合数据。

我们会使用后验预测分布通过我们得到的参数生成一批模拟数据，并将其与真实数据进行对比。

In [57]:
# 后验预测分布的计算仍在容器中进行
with model3:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace3.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace3, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [48]:
# 绘制后验预测分布
az.plot_ppc(trace3)
plt.show()

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


<Figure size 432x288 with 1 Axes>

### (8)模型比较

前面的诊断过程，我们只考虑了模型3 (随机斜率模型)。在模型比较阶段，我们可以同时比较三个模型：
- 模型1：仅包含随机截距的模型，拟合不同学校的平均留级率。
- 模型2：随机截距模型。在模型1的基础上，考虑学前教育在group level的总体效应。
- 模型3：随机斜率模型。在模型2的基础上，考虑学前教育在不同学校上 (population level)的不同效应。

In [53]:
# 对模型进行mcmc采样
with model1:
    trace1 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)
with model2:
    trace2 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)
# 模型3前面完成了采样，这里可以不用再采样了
# with model3:
#     trace2 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [alpha, sigma_alpha, mu_alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 9 seconds.
There were 35 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8267075258814216, but should be close to 0.9. Try to increase the number of tuning steps.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


0, dim: obs_id, 338 =? 338


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma_alpha, mu_alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 11 seconds.
There were 18 divergences after tuning. Increase `target_accept` or reparameterize.
There were 112 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8261495138790597, but should be close to 0.9. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.


0, dim: obs_id, 338 =? 338


In [59]:
# 将三个模型的采样结果进行比较
compare_dict = {
    "仅包含截距的模型": trace1, 
    "随机截距模型": trace2, 
    "随机斜率模型": trace3
    }
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='loo')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
随机斜率模型,0,-171.407607,12.588224,0.000000,6.299495e-01,10.366754,0.000000,False,log
仅包含截距的模型,1,-171.729646,9.871100,0.322040,3.700505e-01,10.053856,1.622771,False,log
随机截距模型,2,-172.239602,10.876575,0.831996,6.661338e-16,10.209219,1.142014,False,log


结果显示， 随机斜率模型的拟合度好于其他两个模型，这表示，在不同学校间学前教育对留级率的影响是不同的。

### (9)统计推断


通过模型比较可以发现，学校所带来的随机效应的影响。如果体现这种影响呐？

通过 arviz提供的函数 `plot_forest()` 可以可视化在不同学校间学前教育所带来的影响。
- 图中左边的编号是 学校ID。
- 94% HDI 是学前教育效应 $\beta$ 在不同学校间的可信区间。
- 通过 94% HDI判断效应是否可信发现，学前教育效应在一些学校间 (比如10418, 20204)存在差异，而在一些学校 (比如10109， 20309)不存在差异。

In [63]:
az.plot_forest(trace3, var_names=["beta"], combined=True)
plt.show()

<Figure size 432x424.8 with 1 Axes>

同样，我们可以通过 `az.summary` 函数查看各参数的具体数值。这里只展示了 beta参数的数值，大家可自行查看 alpha参数的相关统计值。

In [66]:
az.summary(trace3, var_names = ["beta"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],-0.867,0.684,-2.094,0.386,0.035,0.025,376.0,459.0,1.01
beta[1],-0.186,0.579,-1.190,0.951,0.027,0.019,450.0,493.0,1.00
beta[2],-0.611,0.597,-1.659,0.594,0.029,0.020,417.0,1357.0,1.00
beta[3],0.101,0.590,-1.001,1.225,0.026,0.018,509.0,1147.0,1.00
beta[4],-1.142,0.793,-2.613,0.212,0.035,0.025,475.0,1378.0,1.01
beta[5],-0.585,0.816,-2.041,1.001,0.028,0.022,751.0,1255.0,1.00
beta[6],-0.810,0.615,-1.952,0.345,0.024,0.017,670.0,1382.0,1.00
beta[7],-1.028,0.690,-2.348,0.182,0.035,0.025,365.0,1398.0,1.01
beta[8],-0.429,0.706,-1.703,1.026,0.036,0.025,393.0,311.0,1.01
beta[9],-0.785,0.678,-2.136,0.351,0.037,0.027,377.0,352.0,1.00


前面的分析是针对 population level 参数，即随机效应。而我们一般想了解组层面 (group level)的效应是否存在，即固定效应。

组层面的效应对应的参数是 mu_alpha, mu_beta, sigma_alpha, sigma_beta。我们可以通过 `az.summary` 与 `az.plot_posterior` 来查看这些参数。

In [68]:
az.plot_posterior(trace3, var_names=['mu_alpha','mu_beta'])
plt.show()

<Figure size 993.6x331.2 with 2 Axes>

In [69]:
az.summary(trace3, var_names=['mu_alpha','mu_beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_alpha,-1.038,0.312,-1.602,-0.419,0.018,0.013,301.0,725.0,1.01
mu_beta,-0.544,0.384,-1.297,0.153,0.020,0.014,359.0,854.0,1.01


whoops！ 别忘了广义线性模型中链接函数的存在，我们需要把上述的参数值转换到0-1的范围。

可以通过 inv.logit函数进行转换，即 $1 / (1 + exp(-\theta))$。

In [72]:
alpha = 1 / (1 + np.exp(-trace3.posterior["mu_alpha"].mean())).to_pandas()
beta = 1 / (1 + np.exp(-(trace3.posterior["mu_beta"].mean()))).to_pandas()
print(
    "未接受学前教育学生的留级率(%) = ",alpha, 
    "\n 接受学前教育学生的留级率(%) = ", alpha + beta,
    "\n 学前教育对留级率的影响(%) = ", beta
    )

未接受学前教育学生的留级率(%) =  0.25993384308806533 
 接受学前教育学生的留级率(%) =  0.6273514968492069 
 学前教育对留级率的影响(%) =  0.3674176537611415


最后，我们比较，模型3(随机斜率模型)和模型2(随机截距模型)中学前教育组层面的效应是否相同。

可见，两个模型的估计非常类似。但是随机斜率模型比随机截距模型更多地考虑了学前教育效应在不同学校间的差异。

In [73]:
beta1 = 1 / (1 + np.exp(-(trace2.posterior["beta"].mean()))).to_pandas()
beta2 = 1 / (1 + np.exp(-(trace3.posterior["mu_beta"].mean()))).to_pandas()
print(
    "随机截距模型中学前教育的效应(%) = ", beta1, 
    "\n 随机斜率模型中学前教育的效应(%) = ", beta2
    )

随机截距模型中学前教育的效应(%) =  0.3828250424387323 
 随机斜率模型中学前教育的效应(%) =  0.3674176537611415


### 总结

- 本节课学习了分层线性模型的基本概念：包括随机截距与随机斜率、固定效应与随机效应、group level与population level。
- 通过 PyMc 分别拟合随机截距模型与随机斜率模型(包含随机截距)。

### 练习

前面我们定义了各种分层模型，但我们没有演示只包含随机斜率的模型，如下图。

接下来的练习大家可以尝试以下定义只包含随机斜率的贝叶斯广义线性分层模型。

关键在于：
- 只允许斜率 beta 随学习变化。而截距 alpha 不变化。
- 如何定义分层模型的先验以及相应的超先验。

![Image Name](https://cdn.kesci.com/upload/image/rmank74ytm.png?imageView2/0/w/640/h/640)

In [77]:
#加载需要使用的库
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm

np.random.seed(123)  # 随机数种子，确保随后生成的随机数相同

# 加载数据
data = pd.read_csv("./thaieduc.csv")
data.PPED = data.PPED.map({'yes':1,'no':0})

# 将数据分层变量"学校(school)"转换为因子(factor)类型
school_idxs, school = pd.factorize(data.SCHOOLID)
# 定义学校与数据的映射：即标注哪名学生(行)属于哪一所学校
coords = {
    "school": school,
    "obs_id": np.arange(len(school_idxs)),
}

In [79]:
with pm.Model(coords=coords) as model_practice:
    ############################################################################
    # 要求：定义level2学校的 Hyperpriors
    # 提示：通过 mu_beta = pm.Normal(???) 进行定义
    ############################################################################
    mu_beta = pm.Normal("mu_beta", ...) 
    sigma_beta = pm.HalfNormal("sigma_beta", ...)  # 注意这是 HalfNormal 分布
    
    ############################################################################
    # 要求：定义先验
    # 提示：通过 beta = pm.Normal(???) 进行定义。通过 dims="school" 使得 beta随学校变化
    ############################################################################
    beta = pm.Normal('beta', ...)
    alpha = pm.Normal('alpha', ...)

    # 定义数据分层变量"学校(school)"
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    x = pm.Data("x", data.PPED, dims="obs_id")
    # 定义线性模型：p是确定性随机变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha + beta[school_idx]*x)
    # 定义似然函数：Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

In [ ]:
pm.model_to_graphviz(model_practice)

In [81]:
with model_practice:
    trace_practice = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [alpha, beta, sigma_beta, mu_beta]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 11 seconds.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
There were 31 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


0, dim: obs_id, 338 =? 338


In [83]:
az.summary(trace_practice, var_names=['mu_beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_beta,-0.616,0.382,-1.363,0.055,0.012,0.009,964.0,1374.0,1.0


In [82]:
az.plot_posterior(trace_practice, var_names=['mu_beta'])
plt.show()

<Figure size 432x288 with 1 Axes>